In [46]:
import polars as pl
import altair as alt



In [47]:
df = pl.read_csv("../data/memory/memory-sizes.csv")

# Replace 'app' and 'unspecified' (case-insensitive) with 'Runtime'

# first group accross categories 
# then group accross types (.text, .data, .bss)
df.filter(pl.col("board") == "adafruit-feather-nrf52840-sense") \
  .group_by(["environment", "benchmark", "type"]) \
  .agg(pl.sum("size"), pl.first("section_total_size")) \
  .group_by(["environment", "benchmark"]) \
  .agg(pl.sum("size").alias("cosy size"), pl.sum("section_total_size").alias("raw size")) \
  .with_columns((pl.col("cosy size") / pl.col("raw size")).alias("cosy to raw ratio")) \
  .sort("cosy to raw ratio", descending=True)


environment,benchmark,cosy size,raw size,cosy to raw ratio
str,str,i64,i64,f64
"""femto-container""","""xgboost""",42626,32464,1.313024
"""femto-container""","""libud""",37730,33616,1.122382
"""wamr""","""tarfind""",122991,110008,1.118019
"""wamr""","""libud""",123527,110544,1.117446
"""wamr""","""crc_32""",123719,110736,1.117243
…,…,…,…,…
"""micro-bpf""","""libud""",114870,110040,1.043893
"""micropython""","""xgboost""",280710,269228,1.042648
"""micro-bpf""","""xgboost""",121758,116928,1.041307


In [48]:

df.filter(pl.col("board") == "adafruit-feather-nrf52840-sense") \
  .filter(pl.col("environment") == "lua", pl.col("benchmark") == "crc_32") \
  .group_by(["environment", "benchmark", "type"]) \
  .agg(pl.sum("size"), pl.first("section_total_size")) \
  .with_columns((pl.col("size") / pl.col("section_total_size")).alias("ratio"))

environment,benchmark,type,size,section_total_size,ratio
str,str,str,i64,i64,f64
"""lua""","""crc_32""",""".bss""",66508,65996,1.007758
"""lua""","""crc_32""",""".data""",624,null,null
"""lua""","""crc_32""",""".text""",178188,164724,1.081737


In [49]:
df = df.with_columns(
    pl.when(pl.col("category").str.to_lowercase().is_in(["unspecified", "fill", "app"]))
      .then(pl.lit("Runtime"))
      .otherwise(pl.col("category"))
      .alias("category")
).group_by(["board", "environment", "benchmark", "category", "type"]) \
  .agg(pl.sum("size"), pl.first("total_size"), pl.first("section_total_size")) \
.filter(pl.col("board") != "native64")

df

board,environment,benchmark,category,type,size,total_size,section_total_size
str,str,str,str,str,i64,i64,i64
"""adafruit-feather-nrf52840-sens…","""jerryscript""","""xgboost""","""Kernel""",""".data""",158,4241180,null
"""adafruit-feather-nrf52840-sens…","""lua""","""xgboost""","""Runtime""",""".text""",90868,3693681,184964
"""adafruit-feather-nrf52840-sens…","""femto-container""","""libud""","""Runtime""",""".bss""",525,1914171,4892
"""adafruit-feather-nrf52840-sens…","""jerryscript""","""libud""","""Kernel""",""".data""",158,4161628,null
"""adafruit-feather-nrf52840-sens…","""native""","""tarfind""","""Kernel""",""".bss""",4883,1884245,4376
…,…,…,…,…,…,…,…
"""adafruit-feather-nrf52840-sens…","""jerryscript""","""libud""","""Runtime""",""".text""",136029,4161628,177572
"""adafruit-feather-nrf52840-sens…","""micropython""","""libud""","""Runtime""",""".text""",103051,3591441,129900
"""adafruit-feather-nrf52840-sens…","""native""","""md5""","""Kernel""",""".text""",20673,1858005,21656


In [ ]:
HEAP_BENCHMARKS = ["md5", "tarfind"]
HEAP_LESS_BENCHMARKS = ["crc_32", "libud", "xgboost"]

BYTE_LABEL_EXPR = """
(datum.value >= 1000000000) ? format(datum.value / 1000000000, '.0f') + ' GB' : 
(datum.value >= 1000000) ? format(datum.value / 1000000, '.0f') + ' MB' : 
(datum.value >= 1000) ? format(datum.value / 1000, '.0f') + ' KB' : 
format(datum.value, '.0f') + ' B'
"""

sort = ["jerryscript", "micropython", "lua", "micro-bpf", "femto-container", "wamr"]
category_order = ["Kernel", "Runtime", "app", "Application",]

colors = {
    "jerryscript":"#1b9e77" ,
    "micropython": "#d95f02",
    "lua": "#7570b3",
    "micro-bpf": "#e7298a",
    "femto-container": "#66a61e",
    "wamr": "#e6ab02",
    "native": "#a6761d"
}
colors_scale = alt.Scale(domain=list(colors.keys()), range=list(colors.values()))

colors_category = {
    "Kernel": "#1f77b4",
    "Runtime": "#ff7f0e",
    "Application": "#2ca02c",
    "Heap": "#d62728",
}

colors_category_scale = alt.Scale(domain=list(colors_category.keys()), range=list(colors_category.values()))

Y_MAX = 350_000

def rom_chart(data):

     return (alt.Chart(data)
            .mark_bar()
            .encode(
                x=alt.X('environment:N', title='', sort=sort),
                y=alt.Y('size:Q', stack='zero', title='Size (bytes)', axis=alt.Axis(labelExpr=BYTE_LABEL_EXPR), scale=alt.Scale(domain=[0, Y_MAX])),
                color=alt.Color('category:N', title='', sort=category_order, scale=colors_category_scale),
                order=alt.Order("color_category_sort_index:Q"),
                tooltip=[
                    alt.Tooltip('type:N', title='Section'),
                    alt.Tooltip('category:N'),
                    alt.Tooltip('size:Q', title='Binary size (bytes)')
                ]
            )
            .facet(
                column=alt.Column('benchmark:N', title="Static memory usage (ROM)"),
                row = alt.Row('board:N', title="Board")
            )
    )
rom_data = df.filter(pl.col("type").is_in([".text", ".data"]))
heap_data = rom_data.filter(pl.col("benchmark").is_in(HEAP_BENCHMARKS))
heap_less_data = rom_data.filter(pl.col("benchmark").is_in(HEAP_LESS_BENCHMARKS))

rom_chart(heap_less_data).display(scaleFactor=2)
rom_chart(heap_data).display(scaleFactor=2)


alt.FacetChart(...)

alt.FacetChart(...)

In [51]:

chart = (alt.Chart(
        df.filter(pl.col("category") == "Application"))
        .mark_bar()
        .encode(
            x=alt.X('environment:N', title='Environment', sort=sort),
            y=alt.Y('size:Q', stack='zero', title='Size (bytes)', axis=alt.Axis(labelExpr=BYTE_LABEL_EXPR)),
            color=alt.Color('environment:N', title='Environment', scale=colors_scale),
            tooltip=[
                alt.Tooltip('type:N', title='Section'),
                alt.Tooltip('size:Q', title='Binary size (bytes)')
            ]
        )
        .facet(
            column=alt.Column('benchmark:N', title="Code Size"),
            row = alt.Row('board:N', title="Board")
        )
)

chart.display(scaleFactor=2)

alt.FacetChart(...)

In [ ]:
app_code_size = pl.read_csv("../data/memory/application_code_size.csv")

chart = (alt.Chart(app_code_size)
        .mark_bar()
        .encode(
            x=alt.X('environment:N', title='', sort=sort),
            y=alt.Y('size_bytes:Q', title='Size (bytes)', axis=alt.Axis(labelExpr=BYTE_LABEL_EXPR)),
            color=alt.Color('environment:N', title='Environment', scale=colors_scale),
            tooltip=[
                alt.Tooltip('type:N', title='Section'),
                alt.Tooltip('size_bytes:Q', title='Binary size (bytes)')
            ]
        )
        .facet(
            column=alt.Column('benchmark:N', title="Code Size"),
        )
)

chart.display(scaleFactor=2)

alt.FacetChart(...)

In [55]:

Y_MAX = 240_000

def ram_chart(data):

    return (alt.Chart(
        data)
        .mark_bar()
        .encode(
            x=alt.X('environment:N', title='', sort=sort),
            y=alt.Y('size:Q', stack='zero', title='Size (bytes)', axis=alt.Axis(labelExpr=BYTE_LABEL_EXPR), scale=alt.Scale(domain=[0, Y_MAX])),
            color=alt.Color('category:N', title='', sort=category_order, scale=colors_category_scale),
            order=alt.Order("color_category_sort_index:Q"),
            tooltip=[
                alt.Tooltip('type:N', title='Section'),
                alt.Tooltip('category:N'),
                alt.Tooltip('size:Q', title='Binary size (bytes)')
            ]
        )
        .facet(
            column=alt.Column('benchmark:N', title="Static memory usage (RAM)"),
            row = alt.Row('board:N', title="Board")
        )
)

ram_data = df.filter(pl.col("type").is_in([".bss", ".data"]))
heap_data = ram_data.filter(pl.col("benchmark").is_in(HEAP_BENCHMARKS))
heap_less_data = ram_data.filter(pl.col("benchmark").is_in(HEAP_LESS_BENCHMARKS))

ram_chart(heap_less_data).display(scaleFactor=2)
ram_chart(heap_data).display(scaleFactor=2)


alt.FacetChart(...)

alt.FacetChart(...)

In [54]:
# df.filter(pl.col("category") == "Heap", pl.col("environment"))